In [2]:
!which is python

/anaconda3/envs/keras/bin/python


In [5]:
import keras
import numpy as np
import random
import os
import time 
import gc 
import random 
from tqdm._tqdm_notebook import tqdm_notebook as tqdm 
import pandas as pd

In [4]:
audiodata = np.loadtxt('test.txt')


NameError: name 'pd' is not defined

In [9]:
labels = pd.read_csv('y.csv').label.values
audiodata = audiodata.reshape((5435,35,173))


In [10]:
audiodata = audiodata.transpose(0,2,1)
audiodata.shape

(5435, 173, 35)

### 1. Raw Data genneration !

In [17]:
np.random.seed(224)

features = 35; # for each song it has 35 kinds of features
time_steps = 173; # nd related time steps 173 



In [12]:
audiodata.shape,labels.shape

((5435, 173, 35), (5435,))

### 2. Dateset

In [13]:
import tensorflow as tf 
keras.__version__

'2.0.8'

In [14]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

x_train,x_test,y_train,y_test = train_test_split(audiodata, labels, test_size=0.2, random_state=224)
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

### 3. Modeling 
#### 1. RNN

In [18]:
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,Activation, Flatten,GRU,LSTM,TimeDistributed, Dense
from keras.optimizers import Adam

rnn = Sequential()
rnn.add(SimpleRNN(
    units = 32,
    batch_input_shape=(None, time_steps, features),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    unroll=True,))
rnn.add(Dense(64, activation="tanh"))
rnn.add(Dense(10))
rnn.add(Activation('softmax'))
rnn.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 32)                2176      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 4,938
Trainable params: 4,938
Non-trainable params: 0
_________________________________________________________________


In [20]:
BATCH_SIZE = 50
BATCH_INDEX = 0
EPOCH = 10

for epoch in range(EPOCH):
    start_time = time.time()
    for step in tqdm(range(100), disable=False):
        # data shape = (batch_num, steps, inputs/outputs)
        X_batch = x_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
        Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
        cost = rnn.train_on_batch(X_batch, Y_batch)
        BATCH_INDEX += BATCH_SIZE
        BATCH_INDEX = 0 if BATCH_INDEX >= x_train.shape[0] else BATCH_INDEX
    
    cost, accuracy = rnn.evaluate(x_test, y_test, batch_size=y_test.shape[0], verbose=False)
    elapsed_time = time.time() - start_time
    print('Epoch {}/{} \t loss={:.4f} \t accuracy={:.4f}\t time={:.2f}s'.format(epoch+1,EPOCH,cost,accuracy,elapsed_time) )
        


Epoch 1/10 	 loss=0.0821 	 accuracy=0.2843	 time=2.77s



Epoch 2/10 	 loss=0.0794 	 accuracy=0.3109	 time=2.31s



Epoch 3/10 	 loss=0.0823 	 accuracy=0.2916	 time=2.66s



Epoch 4/10 	 loss=0.0814 	 accuracy=0.2622	 time=2.43s



Epoch 5/10 	 loss=0.0810 	 accuracy=0.2907	 time=2.33s



Epoch 6/10 	 loss=0.0797 	 accuracy=0.3054	 time=2.59s



Epoch 7/10 	 loss=0.0799 	 accuracy=0.3027	 time=2.65s



Epoch 8/10 	 loss=0.0802 	 accuracy=0.2916	 time=2.49s



Epoch 9/10 	 loss=0.0804 	 accuracy=0.3211	 time=2.37s



Epoch 10/10 	 loss=0.0806 	 accuracy=0.2981	 time=2.50s


#### 2.GRU

In [21]:
gru = Sequential()
gru.add(GRU(
    units = 32,
    batch_input_shape=(None, time_steps, features),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    unroll=True,))
gru.add(Dense(64, activation="tanh"))
gru.add(Dense(10))
gru.add(Activation('softmax'))
gru.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 32)                6528      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 9,290
Trainable params: 9,290
Non-trainable params: 0
_________________________________________________________________


In [23]:
BATCH_SIZE = 50
BATCH_INDEX = 0
EPOCH = 10

for epoch in range(EPOCH):
    start_time = time.time()
    for step in tqdm(range(100), disable=False):
        # data shape = (batch_num, steps, inputs/outputs)
        X_batch = x_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
        Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
        cost = gru.train_on_batch(X_batch, Y_batch)
        BATCH_INDEX += BATCH_SIZE
        BATCH_INDEX = 0 if BATCH_INDEX >= x_train.shape[0] else BATCH_INDEX
    
    cost, accuracy = gru.evaluate(x_test, y_test, batch_size=y_test.shape[0], verbose=False)
    elapsed_time = time.time() - start_time
    print('Epoch {}/{} \t loss={:.4f} \t accuracy={:.4f}\t time={:.2f}s'.format(epoch+1,EPOCH,cost,accuracy,elapsed_time) )
        

Epoch 1/10 	 loss=0.0830 	 accuracy=0.3017	 time=14.87s


Epoch 2/10 	 loss=0.0797 	 accuracy=0.3192	 time=12.79s


Epoch 3/10 	 loss=0.0786 	 accuracy=0.3339	 time=13.21s


Epoch 4/10 	 loss=0.0772 	 accuracy=0.3505	 time=11.19s


Epoch 5/10 	 loss=0.0766 	 accuracy=0.3569	 time=11.30s


Epoch 6/10 	 loss=0.0760 	 accuracy=0.3661	 time=10.74s


Epoch 7/10 	 loss=0.0755 	 accuracy=0.3625	 time=10.78s


Epoch 8/10 	 loss=0.0758 	 accuracy=0.3661	 time=12.38s


Epoch 9/10 	 loss=0.0755 	 accuracy=0.3983	 time=12.10s


Epoch 10/10 	 loss=0.0749 	 accuracy=0.3937	 time=13.22s


In [39]:
lstm = Sequential()
lstm.add(LSTM(
    units = 128,
    batch_input_shape=(None, time_steps, features),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    return_sequences=False))
lstm.add(Dense(256, activation="relu"))       
lstm.add(Dense(64, activation="relu"))
lstm.add(Dense(10))
lstm.add(Activation('softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['categorical_accuracy'])
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 128)               83968     
_________________________________________________________________
dense_15 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                650       
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
Total params: 134,090
Trainable params: 134,090
Non-trainable params: 0
_________________________________________________________________


In [40]:
BATCH_SIZE = 128
BATCH_INDEX = 0
EPOCH = 100

for epoch in range(EPOCH):
    start_time = time.time()
    for step in tqdm(range(40), disable=False):
        # data shape = (batch_num, steps, inputs/outputs)
        X_batch = x_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
        Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
        cost = lstm.train_on_batch(X_batch, Y_batch)
        BATCH_INDEX += BATCH_SIZE
        BATCH_INDEX = 0 if BATCH_INDEX >= x_train.shape[0] else BATCH_INDEX
    
    cost, accuracy = lstm.evaluate(x_test, y_test, batch_size=y_test.shape[0], verbose=False)
    elapsed_time = time.time() - start_time
    print('Epoch {}/{} \t loss={:.4f} \t accuracy={:.4f}\t time={:.2f}s'.format(epoch+1,EPOCH,cost,accuracy,elapsed_time) )
        

Epoch 1/100 	 loss=1.7641 	 accuracy=0.3523	 time=134.18s


Epoch 2/100 	 loss=1.7056 	 accuracy=0.3753	 time=72.44s


Epoch 3/100 	 loss=1.6398 	 accuracy=0.3763	 time=66.03s


Epoch 4/100 	 loss=1.5967 	 accuracy=0.3809	 time=64.59s


Epoch 5/100 	 loss=1.5575 	 accuracy=0.4177	 time=61.18s


...

Epoch 96/100 	 loss=1.1930 	 accuracy=0.5888	 time=60.93s


Epoch 97/100 	 loss=1.1588 	 accuracy=0.6007	 time=71.09s


Epoch 98/100 	 loss=1.1433 	 accuracy=0.6017	 time=100.21s


Epoch 99/100 	 loss=1.1375 	 accuracy=0.6017	 time=90.19s


Epoch 100/100 	 loss=1.1044 	 accuracy=0.6412	 time=66.55s


### Generally speaking, running time : 
> LSTM > GRU > RNN